# Pandas - Zpracování dat (část 2)

V tomto notebooku se naučíme:
- Používat `loc` k modifikaci dat
- Pracovat s prázdnými hodnotami (`isnull`, `notnull`, `dropna`)
- Lambda funkce
- Metodu `apply`

In [1]:
import pandas as pd

---
## 1. Použití `loc` k modifikaci dat

Metodu `loc` lze použít nejen k filtrování dat, ale také k jejich úpravě.

**Syntaxe:**
```python
df.loc[maska = podminka pro řádky, 'sloupec_k_upravě'] = 'nová_hodnota'
```

Kde:
- **podminka** - Series s hodnotami True/False
- **sloupec_k_upravě** - název sloupce, jehož hodnoty chceme změnit

> **Poznámka:** Pokud chceme upravit všechny řádky, místo podmínky napíšeme `:`. Použití `loc` mění data nevratně - je dobré si před úpravou vytvořit kopii pomocí `df.copy()`.

In [2]:
# Vytvoříme ukázkový DataFrame
data = {
    'date': ['2023-1', '1888-0', '2099-13', '2020-5'],
    'currency': ['zł', 'EUR', 'zł', 'EUR'],
    'value': [100, 50, 200, 75],
    'product_types': ['10pcs.', '10 pcs.', '5pcs.', '10pcs.']
}
df = pd.DataFrame(data)

In [3]:
print(df)

      date currency  value product_types
0   2023-1       zł    100        10pcs.
1   1888-0      EUR     50       10 pcs.
2  2099-13       zł    200         5pcs.
3   2020-5      EUR     75        10pcs.


### Příklad: Změna měny z 'zł' na 'PLN'

In [4]:
df.loc[df['currency'] ==  'zł', 'currency']

0    zł
2    zł
Name: currency, dtype: object

In [5]:
# Změna měny z lokální zkratky 'zł' na mezinárodní 'PLN'
df.loc[df['currency'] ==  'zł', 'currency'] = 'PLN'

In [6]:
df

,date,currency,value,product_types
0,2023-1,PLN,100,10pcs.
1,1888-0,EUR,50,10 pcs.
2,2099-13,PLN,200,5pcs.
3,2020-5,EUR,75,10pcs.


In [8]:
# Kontrola - neměly by zůstat žádné řádky s currency='zł'
df.loc[df['currency']=='zł']

,date,currency,value,product_types


### Otázka k zamyšlení
Proč je důležité před použitím `loc` k úpravě dat vytvořit kopii DataFrame?

### Úloha: Oprav chybu v kódu
Následující kód má za úkol změnit všechny hodnoty ve sloupci 'value' na dvojnásobek. Najdi a oprav chybu.

In [10]:
df_copy = df.copy()

In [11]:
# OPRAV CHYBU:
df.loc['value'] = df['value'] * 2

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 0 to value
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           4 non-null      object 
 1   currency       4 non-null      object 
 2   value          4 non-null      float64
 3   product_types  4 non-null      object 
dtypes: float64(1), object(3)
memory usage: 372.0+ bytes


In [22]:
df.loc['value']

date             NaN
currency         NaN
value            NaN
product_types    NaN
Name: value, dtype: object

In [14]:
df['value'] * 2

0        200.0
1        100.0
2        400.0
3        150.0
value      NaN
Name: value, dtype: float64

In [12]:
df

,date,currency,value,product_types
0,2023-1,PLN,100.0,10pcs.
1,1888-0,EUR,50.0,10 pcs.
2,2099-13,PLN,200.0,5pcs.
3,2020-5,EUR,75.0,10pcs.
value,NaN,NaN,NaN,NaN


In [13]:
df_copy

,date,currency,value,product_types
0,2023-1,PLN,100,10pcs.
1,1888-0,EUR,50,10 pcs.
2,2099-13,PLN,200,5pcs.
3,2020-5,EUR,75,10pcs.


In [19]:
df.loc[:,'value'] = df['value'] * 2

In [20]:
df

,date,currency,value,product_types
0,2023-1,PLN,200.0,10pcs.
1,1888-0,EUR,100.0,10 pcs.
2,2099-13,PLN,400.0,5pcs.
3,2020-5,EUR,150.0,10pcs.
value,NaN,NaN,NaN,NaN


---
## 2. Práce s prázdnými hodnotami

Prázdné hodnoty se v datech objevují často z různých důvodů:
- chyby při zadávání
- chybějící informace (legitimní prázdná hodnota)
- chyby při zpracování (např. problémy s převodem znaků na čísla)

Pandas poskytuje tyto metody pro práci s prázdnými hodnotami:
- `isnull()` / `isna()` - zjistí, zda je hodnota prázdná
- `notnull()` / `notna()` - zjistí, zda hodnota není prázdná
- `dropna()` - odstraní řádky s prázdnými hodnotami

In [ ]:
# Vytvoříme DataFrame s prázdnými hodnotami
# import numpy as np

data_null = {
    'product_types': ['jablka', None, 'hrušky', 'banány', None],
    'product_line': [None, 'ovoce', None, None, 'zelenina'],
    'value': [100, 200, None, 150, 300]
}
df_null = pd.DataFrame(data_null)

In [ ]:
print(df_null)

### Metody `isnull()` / `isna()`

Tyto metody jsou identické (sdílejí i dokumentaci). Vrací True tam, kde je hodnota prázdná.

In [ ]:
# Zobrazení původního obsahu
print(df_null.head())

In [ ]:
# Kontrola, které hodnoty jsou prázdné
print(df_null.isna().head())

In [ ]:
# Kontrola prázdných hodnot pouze ve sloupci product_line
df_null['product_line'].isnull()

### Metoda `dropna()`

Odstraňuje řádky s prázdnými hodnotami z DataFrame.

**Parametry:**
- `how` - kdy odstranit řádek:
  - `'any'` - řádek se odstraní, pokud obsahuje jakoukoliv prázdnou hodnotu
  - `'all'` - řádek se odstraní pouze pokud jsou všechny hodnoty prázdné
- `subset` - seznam sloupců, na základě kterých se řádky mažou

In [ ]:
df_null

In [ ]:
df_null.dropna(subset=['product_line'])

In [ ]:
df_null.dropna(subset=['value'])

### Otázka k zamyšlení
Jaký je rozdíl mezi `how='any'` a `how='all'` v metodě `dropna()`? Kdy bys použil/a kterou variantu?

### Úloha: Doplň kód
Doplň kód tak, aby odstranil řádky, které mají prázdnou hodnotu ve sloupci 'product_types'.

In [ ]:
# DOPLŇ:
df_cleaned = df_null.___

---
## 3. Lambda funkce

Lambda je "mini-funkce" převzatá z jazyka LISP. Má stejný efekt jako běžná funkce, ale se zkrácenou syntaxí:
1. Nemá závorky kolem argumentů
2. Nemá klíčové slovo `return`

Lambda funkce nemá jméno, ale může být přiřazena do proměnné.

In [ ]:
# Klasická implementace funkce
def f(x):
    return x**2

In [ ]:
f(3)

In [ ]:
class(f)

In [ ]:
# Implementace pomocí lambda (místo def píšeme lambda)
# a přiřazení do proměnné
g = lambda x: x**2

In [ ]:
print(g(3))

In [ ]:
# Jednorázové použití lambda
print(
    (lambda x: x**2)(3)
)

### Úloha: Oprav chybu
Následující lambda funkce má počítat součet dvou čísel. Najdi a oprav chybu.

In [ ]:
# OPRAV CHYBU:
soucet = lambda x, y: return x + y

---
## 4. Metoda `apply`

Metoda `apply` umožňuje aplikovat funkci na sloupce nebo řádky DataFrame.

**Parametry:**
- `func` - funkce, kterou chceme použít (může být lambda)
- `axis` - osa, na které se funkce aplikuje:
  - `0` - funkce se aplikuje na všechny sloupce
  - `1` - funkce se aplikuje na řádky

> **Poznámka:** Iterování přes DataFrame pomocí `for` cyklu je neefektivní - používejte raději `apply`.

In [ ]:
# Ukázkový DataFrame
data_apply = {
    'value': [100, 200, 300, 400],
    'province': ['PRAHA', 'BRNO', 'OSTRAVA', 'PLZEŇ']
}
df_apply = pd.DataFrame(data_apply)

In [ ]:
print(df_apply)

### Příklad: Zvýšení hodnoty ve sloupci o 23%

In [ ]:
# Zvýšení čísla ve sloupci value o 23%
df_apply[['value']].apply(lambda x: x*1.23)

### Příklad: Převod na malá písmena

In [ ]:
# Změna písmen na malá ve sloupci province
df_apply['province'].apply(lambda x: x.lower())

### Úloha: Doplň kód
Doplň kód tak, aby převedl všechna písmena ve sloupci 'province' na velká.

In [ ]:
# DOPLŇ:
df_apply['province'].apply(lambda x: x.___())

---
## Cvičení 1: Odstranění chyb

Použij soubor **product_prices_renamed.csv** a pomocí `loc` oprav následující chyby:

1. Ve sloupci **date** se objevila data z roku 1888 - '1888-0', změň hodnotu na 1999-1
2. Ve sloupci **date** se objevila data z roku 2099 - '2099-13', změň hodnotu na 2019-1
3. Ve sloupci **product_types** je překlep - oprav ho. Počet kusů má být '10pcs.'. Zkontroluj, zda byl úkol splněn.
4. Pomocí `loc` převeď hodnoty zadané v `EUR` na `PLN` s kurzem 4.15
5. Odfiltruj z datasetu řádky, kde je cena produktu 3000.

**Tip:** Místo psaní `loc` dvakrát nejdřív vyfiltruj data pro řádky kde **currency** = `EUR` a ulož do proměnné.

> Pamatuj, že `loc` mění data nevratně.

In [ ]:
# Tvůj kód zde:


---
## Cvičení 2: Přidání sloupce

Navazuje na řešení předchozího cvičení. Vytvoř nový sloupec **product** pomocí sloupců **product_types** a **product_line**:

1. Zkontroluj, že sloupce **product_types** a **product_line** jsou komplementární (prázdná hodnota v jednom sloupci znamená neprázdnou hodnotu v druhém)
2. Vytvoř nový sloupec **product** s hodnotami ze sloupce **product_types**: `df['product'] = df['product_types']`
3. Najdi neprázdné hodnoty ve sloupci **product_line** a vlož je do sloupce **product**
4. Zvolenou metodou zkontroluj, zda jsou všechny hodnoty ve sloupci **product** neprázdné
5. Odstraň duplicity z tabulky
6. Pomocí metody `to_csv` ulož data (budeme je používat později), nastav `sep=';'` a `index=False`. Ulož soubor jako `product_prices_cleaned.csv`

**Ukázka volání:**
```python
df.to_csv(
    'filepath',
    sep=';',  # nastavení oddělovače
    index=False
)
```

In [ ]:
# Tvůj kód zde:


---
## Přehled použitých metod a funkcí

| Metoda/Funkce | Popis |
|--------------|-------|
| `df.loc[podminka, sloupec]` | Výběr/úprava dat podle podmínky a sloupce |
| `df.copy()` | Vytvoření kopie DataFrame |
| `df.isnull()` / `df.isna()` | Kontrola, zda jsou hodnoty prázdné (vrací True/False) |
| `df.notnull()` / `df.notna()` | Kontrola, zda hodnoty nejsou prázdné |
| `df.dropna(subset=[...])` | Odstranění řádků s prázdnými hodnotami |
| `df.shape` | Vrací rozměry DataFrame (řádky, sloupce) |
| `lambda x: výraz` | Anonymní funkce pro jednoduché transformace |
| `df.apply(func)` | Aplikace funkce na sloupce nebo řádky |
| `df.drop_duplicates()` | Odstranění duplicitních řádků |
| `df.to_csv(path, sep, index)` | Uložení DataFrame do CSV souboru |

---
## Důležité: Jak metódy mění DataFrame?

### 1. Metody s parametrem `inplace`

Mnoho metod má parametr `inplace`, který určuje, zda se změní původní DataFrame:

**Bez `inplace` (výchozí `inplace=False`):**
- Vrací **nový DataFrame**
- Původní DataFrame zůstává **beze změny**
- Výsledek je třeba uložit do proměnné

```python
df_novy = df.dropna()  # df zůstává nezměněn
df_novy = df.drop_duplicates()  # df zůstává nezměněn
```

**S `inplace=True`:**
- Mění **přímo původní DataFrame**
- Vrací `None` (ne nový DataFrame!)
- Změna je nevratná

```python
df.dropna(inplace=True)  # df se změní přímo
df.drop_duplicates(inplace=True)  # df se změní přímo
```

**Metody s parametrem `inplace`:**
- `dropna()`
- `drop_duplicates()`
- `reset_index()`
- `sort_values()`
- `rename()`
- `fillna()`
- `replace()`

### 2. Metody BEZ parametru `inplace`

**`loc` - vždy mění přímo:**
```python
df.loc[podminka, 'sloupec'] = hodnota  # MĚNÍ PŮVODNÍ df!
```
- Nemá parametr `inplace`
- Vždy mění původní DataFrame nevratně
- Proto je dobré před použitím vytvořit kopii: `df = df.copy()`

**`apply()` - vždy vrací nový objekt:**
```python
vysledek = df.apply(funkce)  # VRACÍ nový DataFrame/Series
```
- Nemá parametr `inplace`
- Vždy vrací nový DataFrame nebo Series
- Původní DataFrame zůstává nezměněn

**Filtrování - vrací nový DataFrame:**
```python
df_filtrovany = df[df['A'] > 5]  # VRACÍ nový DataFrame
```

### 3. Doporučení pro začátečníky

**Bezpečný přístup - explicitní přiřazení:**
```python
df = df.dropna()  # Jasné: pracuješ s novým DataFrame
df = df.drop_duplicates()  # Jasné: vytváříš nový DataFrame
```

**Nebo explicitní `inplace`:**
```python
df.dropna(inplace=True)  # Jasné: měníš původní DataFrame
df.drop_duplicates(inplace=True)  # Jasné: měníš původní DataFrame
```

**Pozor na `loc` - vždy vytvoř kopii před úpravami:**
```python
df_backup = df.copy()  # Záloha před změnami
df.loc[podminka, 'sloupec'] = nova_hodnota  # Mění df!
```